<a href="https://colab.research.google.com/github/alibekk93/PWM-for-Secondary-Structure/blob/main/PWM_for_SS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PWM for SS

## Setup